In [1]:
import torch

# Set the device variable for cude if available, else using standard CPU
# device = 'cuda' if torch.cuda.is_available() else 'cpu' # This is for windows
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu") # For M1 Mac
print(f"Device: {device}")

# This will backback to CPU since the llama index is not supported by MPS for llama 3.1
#%env PYTORCH_ENABLE_MPS_FALLBACK=1

Device: mps


In [2]:
import os
from dotenv import load_dotenv, find_dotenv

# these expect to find a .env file at the directory above the lesson.                                                                                                                     # the format for that file is (without the comment)                                                                                                                                       #API_KEYNAME=AStringThatIsTheLongAPIKeyFromSomeService                                                                                                                                     
def load_env():
    _ = load_dotenv(find_dotenv())

def get_HF_key():
    load_env()
    hf_api_key = os.getenv("HUGGING_FACE_TOKEN")
    return hf_api_key

HF_TOKEN =  get_HF_key()

Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 7


In [3]:
# Jupyter runs sync in the background, but we need async capabilities for modules in this code
import nest_asyncio

nest_asyncio.apply()

# Define model and embeddings

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Create the embeddings
embedding_model_id = "sentence-transformers/all-mpnet-base-v2"

# Create the embeddings
embed_model = HuggingFaceEmbedding(
    model_name=embedding_model_id
)

In [5]:
import glob
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Model name that we want to load from HF
#model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# Setup the cache directory for the model and tokenizer.
#cache_dir = "./model/llama3.1_8b/"
cache_dir = "./model/llama3_8b/"

# Check if the model is already exists. If it does, load the model. 
# If file does not exists, then download from HuggingFace
model_files = glob.glob(cache_dir + "*.safetensors")
tokenizer_files = glob.glob(cache_dir + "tokenizer.json")

if len(model_files)>0:
    print("--Model already exists in the directory. Loading from local directory")
    model = AutoModelForCausalLM.from_pretrained(cache_dir)

else:
    # Load the model and the tokenizer
    # The model is loaded with the specified quantization configuration
    # and the "auto" device mapping for efficient inference
    model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16, # Original precision is float32, but we will convert to float16 for efficiency. MPS doesnt support bfloat16 so changing to float16
            device_map="auto",
            token=HF_TOKEN)


if len(tokenizer_files)>0:
    print("\n--Tokenizer already exists in the directory. Loading from local directory")
    tokenizer = AutoTokenizer.from_pretrained(cache_dir)
else:
    tokenizer = AutoTokenizer.from_pretrained(
            model_id, 
            token=HF_TOKEN)
    


--Model already exists in the directory. Loading from local directory


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]


--Tokenizer already exists in the directory. Loading from local directory


## Load data

In [6]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("data/paul_graham/").load_data()

In [7]:
from llama_index.core import Settings
from llama_index.llms.huggingface import HuggingFaceLLM

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": True},
    tokenizer=tokenizer,
    model=model
)

/Users/mridul/anaconda3/envs/llamaindex/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/mridul/anaconda3/envs/llamaindex/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/mridul/anaconda3/envs/llamaindex/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/mridul/anaconda3/envs/llamaindex/lib/python3.1

## Create PropertyGraphIndex

The following steps occur during the creation of a PropertyGraph:

- PropertyGraphIndex.from_documents(): We load documents into an index.

- Parsing Nodes: The index parses the documents into nodes.

- Extracting Paths from Text: The nodes are passed to an LLM, which is prompted to generate knowledge graph triples (i.e., paths).

- Extracting Implicit Paths: The node.relationships property is used to infer implicit paths.

- Generating Embeddings: Embeddings are generated for each text node and graph node, occurring twice during the process.

In [8]:
from llama_index.core import PropertyGraphIndex

index = PropertyGraphIndex.from_documents(
    documents,
    llm=llm,
    embed_model=embed_model,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting paths from text:   0%|          | 0/71 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pa

In [14]:
index.property_graph_store.save_networkx_graph(name="./kg.html")

In [10]:
# Configure settings
Settings.chunk_size = 1024
Settings.embed_model = embed_model
Settings.llm = llm

## Querying

Querying a property graph index typically involves using one or more sub-retrievers and combining their results. The process of graph retrieval includes:

Selecting Nodes: Identifying the initial nodes of interest within the graph.
Traversing: Moving from the selected nodes to explore connected elements.
By default, two primary types of retrieval are employed simultaneously:

- Synonym/Keyword Expansion: Utilizing an LLM to generate synonyms and keywords derived from the query.

- Vector Retrieval: Employing embeddings to locate nodes within your graph.

Once nodes are identified, you can choose to:

- Return Paths: Provide the paths adjacent to the selected nodes, typically in the form of triples.

- Return Paths and Source Text: Provide both the paths and the original source text of the chunk, if available.

In [11]:
retriever = index.as_retriever(
    include_text=False,  # include source text, default True
)

nodes = retriever.retrieve("What happened at Interleaf and Viaweb?")

for node in nodes:
    print(node.text)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Viaweb -> Charged -> $300 a month for a big store
Viaweb -> Has -> 70 stores
Viaweb -> Developed -> Users control through browsers
Yahoo -> Bought -> Viaweb
Viaweb -> Grows -> 7x a year
Viaweb -> Impact -> Elimination of client software
Viaweb -> Had about 500 -> Stores at the end of 1997
Viaweb -> Developed -> Software running on server
Viaweb -> Founded -> 1995
Viaweb -> Was -> Profitable
Viaweb -> Sold for -> $2 million a month
Viaweb -> Priced at $300 a month -> Big store
Viaweb -> Has -> Growth rate
Viaweb -> Went through -> Near-death experiences
Viaweb -> Impact -> Update software right on server
Viaweb -> Was -> Running company
Viaweb -> Impact -> Elimination of typing into command line on server
Viaweb -> Founded -> 1982
Viaweb -> Is -> Startup
Viaweb -> Low price -> Due to author's lack of knowledge
Viaweb -> Grew -> 7x a year
Viaweb -> Low price -> Competitive advantage
Paul graham -> Was -> Partner
Viaweb -> Had about 70 -> Stores at the end of 1996
Viaweb -> Low price -> C

In [16]:
from IPython.display import Markdown

query_engine = index.as_query_engine(
    include_text=True
)

response = query_engine.query("What happened at Interleaf and Viaweb?")

display(Markdown(f"{response.response}"))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


 The author experimented with a new kind of still life painting after moving back to New York. This represents the author's creative adaptation to his new circumstances, as he was unable to find fulfillment in his new life in California and was desperate to refocus on his passion for painting.